In [ ]:
"""
Este modelo tiene varias opciones de instalación pero no me funcionaron. Esto sí
anduvo en colab, instalando los paquetes manualmente como sigue y usando los
scripts del repo.
"""
!pip install paddlepaddle-gpu
!pip install biopython
!pip install paddlenlp

In [ ]:
import os
!git clone https://github.com/CatIIIIIIII/RNAErnie.git
os.chdir("RNAErnie")
# model weights
!gdown 1MQjtnrtssoF5qAiALakaDDBnQfy0PJC4
!gdown 1Cknx2StFQAm-aQtvDpFud5Ii1qlTCVa8
!mv model_config.json model_state.pdparams output/BERT,ERNIE,MOTIF,PROMPT/checkpoint_final/
# download datasets
!wget https://raw.githubusercontent.com/sinc-lab/sincFold/main/data/ArchiveII.csv

In [ ]:
import paddle
from paddlenlp.transformers import ErnieModel
from tqdm.notebook import tqdm

# ========== Set device
paddle.set_device("gpu")
# ========== RNAErnie Model
rna_ernie = ErnieModel.from_pretrained("output/BERT,ERNIE,MOTIF,PROMPT/checkpoint_final/")

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2024-06-10 01:14:58,464] [    INFO] - Loading configuration file output/BERT,ERNIE,MOTIF,PROMPT/checkpoint_final/model_config.json
[2024-06-10 01:14:58,466] [    INFO] - Loading weights file output/BERT,ERNIE,MOTIF,PROMPT/checkpoint_final/model_state.pdparams
[2024-06-10 01:14:59,109] [    INFO] - Loaded weights file from disk, setting weights to model.
[2024-06-10 01:15:01,289] [ WARNING] - Some weights of the model checkpoint at output/BERT,ERNIE,MOTIF,PROMPT/checkpoint_final/ were not used when initializing ErnieModel: ['cls.predictions.decoder_bias', 'cls.predictions.decoder_weight', 'cls.predictions.layer_norm.bias', 'cls.predictions.layer_norm.weight', 'cls.predictions.transform.bias', 'cls.predictions.transform.weight']
- This IS expected if you are initializing ErnieModel from the checkpoint of a model tra

In [ ]:
import pandas as pd
from rna_ernie import BatchConverter
# ========== Prepare Data
#data = [
#    ("RNA1", "GGGUGCGAUCAUACCAGCACUAAUGCCCUCCUGGGAAGUCCUCGUGUUGCACCCCU"),
#    ("RNA2", "GGGUGUCGCUCAGUUGGUAGAGUGCUUGCCUGGCAUGCAAGAAACCUUGGUUCAAUCCCCAGCACUGCA"),
#    ("RNA3", "CGAUUCNCGUUCCC--CCGCCUCCA"),
#]
# data = "./data/ft/seq_cls/nRC/test.fa"
dataset = pd.read_csv("ArchiveII.csv", index_col="id")
data = [(k, dataset.loc[k].sequence) for k in dataset.index]

# ========== Batch Converter
batch_converter = BatchConverter(k_mer=1,
                                  vocab_path="./data/vocab/vocab_1MER.txt",
                                  batch_size=64,
                                  max_seq_len=512)


# call batch_converter to convert sequences to batch inputs
emb_dict = {}
for names, _, inputs_ids in tqdm(batch_converter(data)):
    with paddle.no_grad():
        # extract sequence embeddings
        embeddings = rna_ernie(inputs_ids)[0].detach()
        for k in range(embeddings.shape[0]):
            L = len(dataset.loc[names[k]].sequence)
            emb_dict[names[k]] = embeddings[k, 1:L+1, :].cpu().numpy()


0it [00:00, ?it/s]

In [ ]:
import h5py
with h5py.File(f'RNAErnie_archiveII.h5', 'w') as hdf:
  for key, value in emb_dict.items():
    hdf.create_dataset(key, data=value)

In [ ]:
!mv RNAErnie_archiveII.csv ../drive/MyDrive/lncRNA/LLM-RNA/embeddings/

mv: cannot stat 'RNAErnie_archiveII.csv': No such file or directory
